This notebook calculates standard error on reference data for each dataset-model pair and each metric.

Running time ~ 45 minutes.

In [1]:
import pickle
import os
import datetime

In [2]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, scipy, time, warnings

import sys
from pathlib import Path

import sklearn
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

from evaluation_framework.pipeline import EvaluationPipeline
from evaluation_framework import steps as ef_steps
from evaluation_framework.datasets import get_datasets_for_pattern
from evaluation_framework.runners import run_on_many_datasets

In [3]:
from methods.EF_std import calculate_std_on_dataset
from methods.utils import calculate_targets

In [4]:
# File folder with results, results will be stored here and loaded from there
results_files_folder = 'experiment_results'
if not os.path.exists(results_files_folder):
    os.makedirs(results_files_folder)

In [5]:
datasets = get_datasets_for_pattern(pattern="", data_dir='datasets')

In [6]:
for dataset in datasets:
    dataset.step_size = 2000    
    dataset.observations_in_chunk = 2000

# EF Functions

In [7]:
monitored_models = [
    'LogisticRegression_',
    'LGBMClassifier_',
    'RandomForestClassifier_',
    'XGB_',
    'FT_Transformer_'
]

In [8]:
experiments = []

for monitored_model in monitored_models:
            experiments.append({
                'method':'DATASET_STD',
                'monitored_model': monitored_model,
            })



In [9]:
def save_results_to_df(dat, method, client_model):
    res_data = dat[0]
    df = pd.DataFrame()

    df['method'] = [method]
    df['dataset'] = res_data['dataset_name']
    df['client_model'] = [client_model]
    
    df['std_accuracy'] = res_data['std_accuracy']
    df['std_roc_auc'] = res_data['std_roc']
    df['std_f1'] = res_data['std_f1']

    
    
    return df
    

In [10]:
experiments

[{'method': 'DATASET_STD', 'monitored_model': 'LogisticRegression_'},
 {'method': 'DATASET_STD', 'monitored_model': 'LGBMClassifier_'},
 {'method': 'DATASET_STD', 'monitored_model': 'RandomForestClassifier_'},
 {'method': 'DATASET_STD', 'monitored_model': 'XGB_'},
 {'method': 'DATASET_STD', 'monitored_model': 'FT_Transformer_'}]

In [11]:
len(experiments), len(datasets)

(5, 250)

In [12]:
%%time
results = []
experiments_done = []
experiments_to_rerun = []

for i, experiment in enumerate(experiments[:]):
    method = experiment['method']
    monitored_model = experiment['monitored_model']
    print("experiment {} out of {}".format(i, len(experiments)))
    for j, dataset in enumerate(datasets[:]):
        print("PROGRESS: " + str((i*len(datasets)+j)/len(experiments)/len(datasets)))
    
        full_name = '_'.join((method, monitored_model))

        calculation_step = ef_steps.Step(description='Calculate STD', 
                     func=calculate_std_on_dataset, args=(monitored_model, 2000, 500)) # custom step

        pipeline = EvaluationPipeline(steps=[
            ef_steps.SplitDataStep(), #split raw to train test (reference/analysis)
            ef_steps.CombineProcessedReferenceProductionWithRawStep(), # combine after processing
            ef_steps.SplitIntoChunksStep(), # split to chunks
            ef_steps.Step(description='Calculate targets', 
                     func=calculate_targets, args=(monitored_model, )), # custom step
            calculation_step,
            ])

        res, dat = run_on_many_datasets(datasets=[dataset], pipeline=pipeline, log_metrics=False,)
        
        
        if "traceback" in dat[0].keys():
            experiments_to_rerun.append((experiment, dataset.data_path, dat[0]['traceback']))
        else:
            df_results = save_results_to_df(dat, method, monitored_model)
            results.append(df_results)
            
        experiments_done.append(experiment)


experiment 0 out of 5
PROGRESS: 0.0

Evaluating on: datasets\employment-2015_2016_2017_2018-AK.pq
Chunk size: 2000, all chunks: 10, reference_chunks: 3, transition chunks: 1, production_chunks (includes transition) :7
PROGRESS: 0.0008

Evaluating on: datasets\employment-2015_2016_2017_2018-AL.pq
Chunk size: 2000, all chunks: 71, reference_chunks: 23, transition chunks: 1, production_chunks (includes transition) :48
PROGRESS: 0.0016

Evaluating on: datasets\employment-2015_2016_2017_2018-AR.pq
Chunk size: 2000, all chunks: 45, reference_chunks: 14, transition chunks: 1, production_chunks (includes transition) :31
PROGRESS: 0.0024

Evaluating on: datasets\employment-2015_2016_2017_2018-AZ.pq
Chunk size: 2000, all chunks: 103, reference_chunks: 34, transition chunks: 1, production_chunks (includes transition) :69
PROGRESS: 0.0032

Evaluating on: datasets\employment-2015_2016_2017_2018-CA.pq
Chunk size: 2000, all chunks: 566, reference_chunks: 188, transition chunks: 1, production_chunks (

PROGRESS: 0.032799999999999996

Evaluating on: datasets\employment-2015_2016_2017_2018-TN.pq
Chunk size: 2000, all chunks: 100, reference_chunks: 32, transition chunks: 1, production_chunks (includes transition) :68
PROGRESS: 0.033600000000000005

Evaluating on: datasets\employment-2015_2016_2017_2018-TX.pq
Chunk size: 2000, all chunks: 397, reference_chunks: 130, transition chunks: 1, production_chunks (includes transition) :267
PROGRESS: 0.0344

Evaluating on: datasets\employment-2015_2016_2017_2018-UT.pq
Chunk size: 2000, all chunks: 46, reference_chunks: 14, transition chunks: 1, production_chunks (includes transition) :32
PROGRESS: 0.0352

Evaluating on: datasets\employment-2015_2016_2017_2018-VA.pq
Chunk size: 2000, all chunks: 126, reference_chunks: 41, transition chunks: 1, production_chunks (includes transition) :85
PROGRESS: 0.036

Evaluating on: datasets\employment-2015_2016_2017_2018-VT.pq
Chunk size: 2000, all chunks: 9, reference_chunks: 3, transition chunks: 1, productio

PROGRESS: 0.0664

Evaluating on: datasets\income-2015_2016_2017_2018-NY.pq
Chunk size: 2000, all chunks: 153, reference_chunks: 50, transition chunks: 1, production_chunks (includes transition) :103
PROGRESS: 0.06720000000000001

Evaluating on: datasets\income-2015_2016_2017_2018-OH.pq
Chunk size: 2000, all chunks: 92, reference_chunks: 30, transition chunks: 1, production_chunks (includes transition) :62
PROGRESS: 0.068

Evaluating on: datasets\income-2015_2016_2017_2018-OK.pq
Chunk size: 2000, all chunks: 26, reference_chunks: 8, transition chunks: 1, production_chunks (includes transition) :18
PROGRESS: 0.0688

Evaluating on: datasets\income-2015_2016_2017_2018-OR.pq
Chunk size: 2000, all chunks: 32, reference_chunks: 10, transition chunks: 1, production_chunks (includes transition) :22
PROGRESS: 0.0696

Evaluating on: datasets\income-2015_2016_2017_2018-PA.pq
Chunk size: 2000, all chunks: 100, reference_chunks: 33, transition chunks: 1, production_chunks (includes transition) :67
P

PROGRESS: 0.0992

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-MS.pq
Chunk size: 2000, all chunks: 18, reference_chunks: 6, transition chunks: 1, production_chunks (includes transition) :12
PROGRESS: 0.1

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-MT.pq
Chunk size: 2000, all chunks: 5, reference_chunks: 1, transition chunks: 1, production_chunks (includes transition) :4
PROGRESS: 0.1008

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-NC.pq
Chunk size: 2000, all chunks: 56, reference_chunks: 19, transition chunks: 1, production_chunks (includes transition) :37
PROGRESS: 0.1016

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-ND.pq
Chunk size: 2000, all chunks: 3, reference_chunks: 1, transition chunks: 1, production_chunks (includes transition) :2
PROGRESS: 0.1024

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-NE.pq
Chunk size: 2000, all chunks: 9, reference_chunks: 3, transition chunks: 1, production_chunks (includes transition) :6
PROGRESS

PROGRESS: 0.132

Evaluating on: datasets\mobility-2015_2016_2017_2018-KS.pq
Chunk size: 2000, all chunks: 8, reference_chunks: 2, transition chunks: 1, production_chunks (includes transition) :6
PROGRESS: 0.1328

Evaluating on: datasets\mobility-2015_2016_2017_2018-KY.pq
Chunk size: 2000, all chunks: 12, reference_chunks: 4, transition chunks: 1, production_chunks (includes transition) :8
PROGRESS: 0.1336

Evaluating on: datasets\mobility-2015_2016_2017_2018-LA.pq
Chunk size: 2000, all chunks: 12, reference_chunks: 4, transition chunks: 1, production_chunks (includes transition) :8
PROGRESS: 0.13440000000000002

Evaluating on: datasets\mobility-2015_2016_2017_2018-MA.pq
Chunk size: 2000, all chunks: 22, reference_chunks: 7, transition chunks: 1, production_chunks (includes transition) :15
PROGRESS: 0.1352

Evaluating on: datasets\mobility-2015_2016_2017_2018-MD.pq
Chunk size: 2000, all chunks: 16, reference_chunks: 5, transition chunks: 1, production_chunks (includes transition) :11
PR

...

PROGRESS: 0.8528

Evaluating on: datasets\income-2015_2016_2017_2018-KY.pq
Chunk size: 2000, all chunks: 32, reference_chunks: 10, transition chunks: 1, production_chunks (includes transition) :22
PROGRESS: 0.8536

Evaluating on: datasets\income-2015_2016_2017_2018-LA.pq
Chunk size: 2000, all chunks: 31, reference_chunks: 10, transition chunks: 1, production_chunks (includes transition) :21
PROGRESS: 0.8543999999999999

Evaluating on: datasets\income-2015_2016_2017_2018-MA.pq
Chunk size: 2000, all chunks: 59, reference_chunks: 19, transition chunks: 1, production_chunks (includes transition) :40
PROGRESS: 0.8552000000000001

Evaluating on: datasets\income-2015_2016_2017_2018-MD.pq
Chunk size: 2000, all chunks: 49, reference_chunks: 16, transition chunks: 1, production_chunks (includes transition) :33
PROGRESS: 0.856

Evaluating on: datasets\income-2015_2016_2017_2018-ME.pq
Chunk size: 2000, all chunks: 10, reference_chunks: 3, transition chunks: 1, production_chunks (includes transitio

PROGRESS: 0.8856

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-DE.pq
Chunk size: 2000, all chunks: 4, reference_chunks: 1, transition chunks: 1, production_chunks (includes transition) :3
PROGRESS: 0.8864

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-FL.pq
Chunk size: 2000, all chunks: 108, reference_chunks: 36, transition chunks: 1, production_chunks (includes transition) :72
PROGRESS: 0.8872000000000001

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-GA.pq
Chunk size: 2000, all chunks: 57, reference_chunks: 19, transition chunks: 1, production_chunks (includes transition) :38
PROGRESS: 0.888

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-HI.pq
Chunk size: 2000, all chunks: 7, reference_chunks: 2, transition chunks: 1, production_chunks (includes transition) :5
PROGRESS: 0.8887999999999999

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-IA.pq
Chunk size: 2000, all chunks: 15, reference_chunks: 5, transition chunks: 1, production_chunks (inc

PROGRESS: 0.9184

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-WV.pq
Chunk size: 2000, all chunks: 10, reference_chunks: 3, transition chunks: 1, production_chunks (includes transition) :7
PROGRESS: 0.9192

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-WY.pq
could not execute step Split preprocessed data into chunks that will be points for nanny model: index 0 is out of bounds for axis 0 with size 0
PROGRESS: 0.92

Evaluating on: datasets\mobility-2015_2016_2017_2018-AK.pq
could not execute step Split preprocessed data into chunks that will be points for nanny model: index -1 is out of bounds for axis 0 with size 0
PROGRESS: 0.9208

Evaluating on: datasets\mobility-2015_2016_2017_2018-AL.pq
Chunk size: 2000, all chunks: 12, reference_chunks: 4, transition chunks: 1, production_chunks (includes transition) :8
PROGRESS: 0.9216

Evaluating on: datasets\mobility-2015_2016_2017_2018-AR.pq
Chunk size: 2000, all chunks: 8, reference_chunks: 2, transition chunks: 1, producti

PROGRESS: 0.9512

Evaluating on: datasets\mobility-2015_2016_2017_2018-SC.pq
Chunk size: 2000, all chunks: 13, reference_chunks: 4, transition chunks: 1, production_chunks (includes transition) :9
PROGRESS: 0.952

Evaluating on: datasets\mobility-2015_2016_2017_2018-SD.pq
could not execute step Split preprocessed data into chunks that will be points for nanny model: index -1 is out of bounds for axis 0 with size 0
PROGRESS: 0.9528

Evaluating on: datasets\mobility-2015_2016_2017_2018-TN.pq
Chunk size: 2000, all chunks: 18, reference_chunks: 6, transition chunks: 1, production_chunks (includes transition) :12
PROGRESS: 0.9536

Evaluating on: datasets\mobility-2015_2016_2017_2018-TX.pq
Chunk size: 2000, all chunks: 79, reference_chunks: 26, transition chunks: 1, production_chunks (includes transition) :53
PROGRESS: 0.9544

Evaluating on: datasets\mobility-2015_2016_2017_2018-UT.pq
Chunk size: 2000, all chunks: 10, reference_chunks: 3, transition chunks: 1, production_chunks (includes tra

PROGRESS: 0.984

Evaluating on: datasets\traveltime-2015_2016_2017_2018-NJ.pq
Chunk size: 2000, all chunks: 63, reference_chunks: 20, transition chunks: 1, production_chunks (includes transition) :43
PROGRESS: 0.9848

Evaluating on: datasets\traveltime-2015_2016_2017_2018-NM.pq
Chunk size: 2000, all chunks: 11, reference_chunks: 3, transition chunks: 1, production_chunks (includes transition) :8
PROGRESS: 0.9856

Evaluating on: datasets\traveltime-2015_2016_2017_2018-NV.pq
Chunk size: 2000, all chunks: 19, reference_chunks: 6, transition chunks: 1, production_chunks (includes transition) :13
PROGRESS: 0.9863999999999999

Evaluating on: datasets\traveltime-2015_2016_2017_2018-NY.pq
Chunk size: 2000, all chunks: 135, reference_chunks: 44, transition chunks: 1, production_chunks (includes transition) :91
PROGRESS: 0.9872000000000001

Evaluating on: datasets\traveltime-2015_2016_2017_2018-OH.pq
Chunk size: 2000, all chunks: 80, reference_chunks: 26, transition chunks: 1, production_chunks 

In [13]:
df = pd.concat(results).reset_index(drop=True)

In [14]:
df.head()

,method,dataset,client_model,std_accuracy,std_roc_auc,std_f1
0,DATASET_STD,employment-2015_2016_2017_2018-AK.pq,LogisticRegression_,0.007524,0.006391,0.009545
1,DATASET_STD,employment-2015_2016_2017_2018-AL.pq,LogisticRegression_,0.008247,0.006935,0.010766
2,DATASET_STD,employment-2015_2016_2017_2018-AR.pq,LogisticRegression_,0.008216,0.006937,0.010643
3,DATASET_STD,employment-2015_2016_2017_2018-AZ.pq,LogisticRegression_,0.008628,0.006971,0.011151
4,DATASET_STD,employment-2015_2016_2017_2018-CA.pq,LogisticRegression_,0.009382,0.008207,0.010505


In [16]:
results_files_folder

'experiment_results'

In [17]:
df.to_parquet(os.path.join(results_files_folder, 'STD_results.pq'))
